<a href="https://colab.research.google.com/github/Mubashar-Bashir/AgenticAI/blob/main/context_openai_agents_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install openai-agents SDK

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.8 MB/s eta 0:00:00


# Make your Notebook capable of running asynchronous functions.
Both Jupyter notebooks and Python’s asyncio library utilize event loops, but they serve different purposes and can sometimes interfere with each other.

The nest_asyncio library allows the existing event loop to accept nested event loops, enabling asyncio code to run within environments that already have an event loop, such as Jupyter notebooks.

In summary, both Jupyter notebooks and Python’s asyncio library utilize event loops to manage asynchronous operations. When working within Jupyter notebooks, it’s essential to be aware of the existing event loop to effectively run asyncio code without conflicts.

In [2]:
import nest_asyncio
nest_asyncio.apply()

# Config

In [3]:
from pydantic import BaseModel
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)
from google.colab import userdata


In [4]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

In [5]:
from agents import set_default_openai_client, set_tracing_disabled
set_default_openai_client(external_client)
set_tracing_disabled(True)

# Context Management


Context available locally to your code: this is data and dependencies you might need when tool functions run, during callbacks like on_handoff, in lifecycle hooks, etc.
- You create any Python object you want. A common pattern is to use a dataclass or a Pydantic object.
- You pass that object to the various run methods (e.g. Runner.run(..., **context=whatever**)).
- All your tool calls, lifecycle hooks etc will be passed a wrapper object, - RunContextWrapper[T], where T represents your context object type which you can access via wrapper.context.
- The most important thing to be aware of: every agent, tool function, lifecycle etc for a given agent run must use the same type of context.

You can use the context for things like:
- Contextual data for your run (e.g. things like a username/uid or other information about the user)
- Dependencies (e.g. logger objects, data fetchers, etc)
- Helper functions

[Learning Reference](https://openai.github.io/openai-agents-python/context/)

# Local Context

In [21]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo1:
    name: str
    uid: int
    cell:str | None
    Address: str|None
    Account_Number:str = "ALF12345563"
    location: str = "Pakistan"
    balance: float = 0.0


@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo1]) -> str:
    '''Returns the age and name of the user.'''
    print("wrapper User Age >>> ",wrapper)
    return f"User {wrapper.context.name} is 30 years old"

@function_tool
async def fetch_user_location(wrapper: RunContextWrapper[UserInfo1]) -> str:
    '''Returns the location of the user.'''
    print("wrapper Location >>> ",wrapper)
    return f"User {wrapper.context.name} is from {wrapper.context.location}"
#to add for user payment to his account
@function_tool
async def user_payment(wrapper: RunContextWrapper[UserInfo1], amount: float) -> str:
    '''Adds the specified amount to the user's account balance.'''
    print("wrapper Payment >>> ",wrapper.context.balance)
    print("Amount recived>>>", amount)

    # wrapper.context.Account_Number += float(amount)
    print(f" Account number = {wrapper.context.Account_Number}")
    wrapper.context.balance += float(amount)
# show the balance Amount of user
@function_tool
async def check_balance(wrapper: RunContextWrapper[UserInfo1]) -> str:
    '''Returns the current account balance of the user.'''
    return f"User {wrapper.context.name}'s account balance is {wrapper.context.balance} Rupees."
#create

async def main():
    user_info = UserInfo1(name="Muhammad Qasim", uid=123, location="KHI", cell="03446062616",Address="GRW")

    agent = Agent[UserInfo1](
        name="Assistant",
        instructions="Assistant can help user to add Balance in the said Account",
        tools=[fetch_user_age,fetch_user_location,user_payment,check_balance],
        model=model
    )

    result = await Runner.run(
        starting_agent=agent,
        input="Add 5Lac Rupees to the Muhammad Qasim Account , show the balance in the account",
        context=user_info,
    )

    print(result.final_output)
    # The user John is 47 years old.

if __name__ == "__main__":
    asyncio.run(main())

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79cebf33aff0>


...

LLM resp:
{
  "content": "OK. I've added 500000.0 Rupees to the user's account. The current balance is 500000.0 Rupees.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "OK. I've added 500000.0 Rupees to the user's account. The current balance is 500000.0 Rupees.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


OK. I've added 500000.0 Rupees to the user's account. The current balance is 500000.0 Rupees.


# Agent/LLM context

In [11]:
from agents import enable_verbose_stdout_logging

enable_verbose_stdout_logging()

In [12]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:
    name: str
    uid: int
    cell:str
    Address: str
    Account_Number:str

@function_tool
async def greet_user(context: RunContextWrapper[UserInfo], greeting: str) -> str:
  """Greets the User with their name.
  Args:
    greeting: A specialed greeting message for user
  """
  name = context.context.name
  return f"Hello {name}, {greeting}"

async def main():
    user_info = UserInfo(name="Muhammad Qasim", uid=123)

    agent = Agent[UserInfo](
        name="Assistant",
        tools=[greet_user],
        model=model,
        # Dynamic function context pass in instructions/system prompmt/developer prompt
        instructions="Always greet the user using <function_call>greet_user</function_call> and welcome them to Panaversity"
    )

    result = await Runner.run(
        starting_agent=agent,
        input="Hello",
        context=user_info,
    )

    print(result.final_output)

asyncio.run(main())

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ced2bc8710>


...

LLM resp:
{
  "content": "Hello Muhammad Qasim, Welcome to Panaversity!\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "Hello Muhammad Qasim, Welcome to Panaversity!\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


Hello Muhammad Qasim, Welcome to Panaversity!



In [ ]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:
    name: str
    uid: int

@function_tool
async def greet_user(context: RunContextWrapper[UserInfo], greeting: str) -> str:
  """Greets the User with their name.
  Args:
    greeting: A specialed greeting message for user
  """
  name = context.context.name
  return f"Hello {name}, {greeting}"

async def main():
    user_info = UserInfo(name="Junaid", uid=123)

    agent = Agent[UserInfo](
        name="Assistant",
        tools=[greet_user],
        model=model,
        instructions="Always greet the user using <function_call>greet_user</function_call> and welcome them to Panaversity"
    )

    result = await Runner.run(
        starting_agent=agent,
        input="Hello",
        context=user_info,
    )

    print(result.final_output)

asyncio.run(main())

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x78b2cb02ce90>


...

LLM resp:
{
  "content": "Hello Junaid, Welcome to Panaversity!\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "Hello Junaid, Welcome to Panaversity!\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


Hello Junaid, Welcome to Panaversity!

